In [1]:
import pymysql
import redis
import time
import re
import datetime

In [22]:
r = redis.StrictRedis(host='192.168.1.176', port=6379,decode_responses=True)

In [11]:
def connect_to_db(DB_NAME):
    # 設定資料庫連線資訊
    host = '192.168.1.52'
    port = 3306
    user = 'recipe'
    passwd = 'recipe'
    db = DB_NAME
    charset = 'utf8mb4'

    # 建立連線
    conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db, charset=charset)
    print('Successfully connected to DB : {} !'.format(DB_NAME))
    return conn

In [18]:
db = 'recipe'
conn = connect_to_db(db)  # 連接資料庫
cursor = conn.cursor()

Successfully connected to DB : recipe !


In [17]:
cursor

In [15]:
    cursor.close()  
    conn.close()

In [23]:
# scenario 1. ingredient storage record (when kafka consumer get data)

def ingredient_storage(kafka_data,line_id):

    db = 'recipe'
    conn = connect_to_db(db)  # 連接資料庫
    cursor = conn.cursor()


    uid = r.hget('total_user_id',line_id)  #從line_id中找user_id


    food_data = kafka_data.split()   #['壽司飯', '200', 'g']

    if not r.hexists("general_ingredient", food_data[0]):  #如果未在general食材的話,換成general食材
        food_data[0] = r.hget('synonym',food_data[0])
        if food_data[0]:
            pass
        else:
            return "查無相關食材名稱，請重新輸入"
    food_id = r.hget('general_ingredient',food_data[0])    #查出食材ID


    t = time.localtime()     #食材存進的時間
    dt = time.strftime('%Y-%m-%d', t)


    #到期日
    sql_get_storage_time = """
    SELECT 保存期限 FROM ingredient WHERE 食材ID = %s; 
    """
    cursor.execute(sql_get_storage_time % food_id )
    storage_time = cursor.fetchall()
    regex = re.compile(r'\d+')
    match = regex.search(storage_time[0][0])
    expire_t = (datetime.datetime.now()+datetime.timedelta(days=int(match.group()))).strftime("%Y-%m-%d")



    sql = """
    INSERT INTO refrigerator_record (使用者ID, 食材ID, 食材重量, 食材單位, 食材存放日, 食材到期日)
    VALUES ('{}', '{}', '{}', '{}', '{}', '{}');
    """.format(uid,food_id,food_data[1],food_data[2],dt,expire_t)

    cursor.execute(sql)  
    conn.commit()

    cursor.close()  
    conn.close()
    
    
    r.hset(line_id,food_data[0],(food_data[1] + ',' + food_data[2] + ',' + dt)) #寫進redis
    
    return(print("已寫入",food_data,"進",uid,"冰箱"))

In [24]:
# scenario 1. ingredient storage record (when kafka consumer get data)

kafka_data = '雞蛋 200 g'  #input冰箱的食材
line_id = 'U429ec102b46a5332b32c4f1a8b3b04db'
ingredient_storage(kafka_data,line_id)

Successfully connected to DB : recipe !
已寫入 ['雞蛋', '200', 'g'] 進 user1 冰箱


In [ ]:
# scenario 2. user profile + line_id (when questioner completed)
db = 'recipe'
conn = connect_to_db(db)  
cursor = conn.cursor()

try:

    sql_to_profile = """
        INSERT INTO refrigerator_record (使用者ID,Line_ID,姓名,email,性別,電話,生日)  
        VALUES ('{}', '{}', '{}', '{}', '{}', '{}');
        """.format(問卷格式待抓)

    sql_to_my_own_preference = """
        INSERT INTO my_own_prefernece (使用者ID,偏好ID)
        VALUES ('{}', '{}', '{}', '{}', '{}', '{}');
        """.format(問卷格式待抓)

    cursor.execute(sql_to_profile)  
    conn.commit()
    cursor.execute(sql_to_my_own_preference)  
    conn.commit()

    cursor.close()  
    conn.close()

    r.hset('total_user_id',line_id,uid)  #新成員加入hash →  'total_user_id': {line_ID:userID} 供未來直接用lineID可查userID
    
except: 
    print('帳號重複')

In [191]:
 # scenario 3. user like recipe record (when user has ordered a recipe)

line_id = '00'
recipe_id = 900
deduct_ingredient_from_icebox(line_id,recipe_id)

Successfully connected to DB : recipe !
須扣除的食材 ['地瓜', '麵粉']
已將 ['地瓜', '麵粉'] 從 user7 冰箱扣除


In [190]:
 # scenario 3. user like recipe record (when user has ordered a recipe)
def deduct_ingredient_from_icebox (line_id,recipe_id):
    db = 'recipe'
    conn = connect_to_db(db) 
    cursor = conn.cursor()
    
    uid = r.hget('total_user_id',line_id)  #從line_id中找user_id
    
    t = time.localtime()     #食材取用的時間
    dt = time.strftime('%Y-%m-%d', t)
    
    r_ing = r.hget(recipe_id,'ingredient')  #找出指定食譜的食材  →'紅蘿蔔 260 gram, 豬肉 110 gram'
    r_ing = r_ing. split(',')
    recipe_ing_set = set()
    for each in r_ing:
        b = each.split(' ')
        recipe_ing_set.add(b[0])
        
    icebox_ing = set(r.hkeys(line_id))   #找出user冰箱的的食材
    ing_intersection = list(recipe_ing_set & icebox_ing)   # 找出食譜裡有的冰箱食材 (須扣除的食材→ ['紅蘿蔔', '豬肉'])
    food_id = r.hmget('general_ingredient',ing_intersection)   #查出扣除食材id →[30,49]  
    print("須扣除的食材",ing_intersection)
        
    
    for each_ing_id in range(len(food_id)):     #每一食材id: mysql 更新, redis 刪除 
        sql = """
        UPDATE refrigerator_record SET 食材取用日 = "{}" WHERE 使用者ID = "{}" AND 食材ID = "{}";
        """.format(dt,uid,food_id[each_ing_id])

        cursor.execute(sql)  
        conn.commit()
    
        r.hdel(line_id,ing_intersection[each_ing_id])
    
    
    #食譜紀錄增加
    sql_my_recipe_record = """                   
    INSERT INTO my_recipe_record (使用者ID, 食譜ID, 食譜使用日期) 
    VALUES ('{}', '{}', '{}');
    """.format(uid,recipe_id,dt)
        
    cursor.execute(sql_my_recipe_record)  
    conn.commit()
        
    cursor.close()  
    conn.close()
    return(print("已將",ing_intersection,"從",uid,"冰箱扣除"))

In [187]:
r_ing = r.hget('900','ingredient')
r_ing = r_ing. split(',')
recipe_ing_set = set()
for each in r_ing:
    b = each.split(' ')
    recipe_ing_set.add(b[0])
recipe_ing_set

{'地瓜', '太白粉', '玉米粉', '蝦', '麵粉'}

In [188]:
icebox_ing = set(r.hkeys('00'))
icebox_ing

{'九層塔', '地瓜', '山蘇', '檸檬', '牛奶', '牛肉', '脆瓜', '芋頭', '蔥', '蘋果', '雞蛋', '麵粉'}

In [189]:
ing_intersection = list(recipe_ing_set & icebox_ing)
ing_intersection

['地瓜', '麵粉']

In [130]:
food_id = r.hmget('general_ingredient',ing_intersection) 
food_id 
food_id[0]

'30'

In [154]:
line_id = '00'
recipe_id = 15000

In [157]:
db = 'recipe'
conn = connect_to_db(db) 
cursor = conn.cursor()

t = time.localtime()     #食材存進的時間
dt = time.strftime('%Y-%m-%d', t)

sql_my_recipe_record = """
INSERT INTO my_recipe_record (使用者ID, 食譜ID, 食譜使用日期)
VALUES ('{}', '{}', '{}');
""".format(uid,recipe_id,dt)

cursor.execute(sql_my_recipe_record)  
conn.commit()

cursor.close()  
conn.close()

Successfully connected to DB : recipe !
